In [14]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Import Python Module
from AnimalShelter_CRUD_Dash_App import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "makecake"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)
# Read Database
df = pd.DataFrame.from_records(db.read({}))
# Drop Column
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Retrieve Image File
image_filename = 'Salvare.png' 
# Read Image File
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    html.Div(dcc.RadioItems(id='filter-type',
            options=[
             {'label': 'Water Rescue', 'value': 'WR'},
             {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
             {'label': 'Desaster or Individual Tracking', 'value': 'DIT'},
             {'label': 'RESET', 'value': 'R'}
                    ],
            value='WR'
             
    )),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'), 
                         style_table={'overflowX':'auto'},
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         style_header={'backgroundColor': 'red', 'fontWeight': 'bold'},
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.H3("Greatness is rarely acheived")
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(value):

        if (value == 'WR'):
            dff = pd.DataFrame.from_records(db.read({"$and":[
                {'sex_upon_outcome':'Intact Female'},
                {"$or":[{'breed': 'Labrador Retriever Mix'},
                {'breed': 'Chesapeake Bay Retriever'},
                {'breed':'Newfoundland Mix'}]},
                {'$or': [
                    {'age_upon_outcome_in_weeks': {'$gt': 2, '$lt': 156}}]
                }]}))
            dff.drop(columns=['_id'],inplace=True)    #Archimedes
            data=dff.to_dict('records')
            return data
        elif (value == 'MWR'):
            dff = pd.DataFrame.from_records(db.read({"$and":[
                {'sex_upon_outcome':'Intact Male'},
                {"$or":[{'breed': 'German Sheperd'},
                {'breed': 'Alaskan Malamute'},
                {'breed':'Old English Sheepdog'},
                {'breed':'Siberian Husky'},
                {'breed':'Rottweiler'}]},
                {'$or': [
                    {'age_upon_outcome_in_weeks': {'$gt': 2, '$lt': 156}}]
                }]}))
            dff.drop(columns=['_id'],inplace=True)    #Archimedes
            data=dff.to_dict('records')
            return data  
        elif (value == 'DIT'):
            dff = pd.DataFrame.from_records(db.read({"$and":[
                {'sex_upon_outcome':'Intact Male'},
                {"$or":[{'breed': 'Doberman Pinscher'},
                {'breed': 'German Sheperd'},
                {'breed':'Golden Retriever'},
                {'breed':'Bloodhound'},
                {'breed':'Rottweiler'}]},
                {'$or': [
                    {'age_upon_outcome_in_weeks': {'$gt': 20, '$lt': 300}}]
                }]}))
            dff.drop(columns=['_id'],inplace=True)    #Archimedes
            data=dff.to_dict('records')
            return data  
        else:
            dff = pd.DataFrame.from_records(db.read({}))
            dff = dff[dff['breed'].isin(['Doberman Pinscher', 'German Sheperd', 'Golden Retriever', 'Bloodhound', 'Rottweiler', 'Alaskan Malamute',
                'Old English Sheepdog', 'Siberian Husky', 'Chesapeake Bay Retriever', 'Newfoundland Mix'])]
            dff.drop(columns=['_id'],inplace=True)
            data=dff.to_dict('records')
            return data  
         

    
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_records(db.read({}))
    dff.drop(columns=['_id'],inplace=True)
    
    if viewData is None:
        dff = dff[dff['breed'].isin(['Doberman Pinscher', 'German Sheperd', 'Golden Retriever', 'Bloodhound', 'Rottweiler', 'Alaskan Malamute',
                'Old English Sheepdog', 'Siberian Husky', 'Chesapeake Bay Retriever', 'Newfoundland Mix'])]
        dff = dff.breed.value_counts()
        dff.name = "breed"
        dff = dff.to_frame()
        return [
        dcc.Graph(            
            figure = (px.pie(dff, names='breed', title='Preferred Animals')).update_layout(autosize=False, width=50, height=50)
        )    
    ]
    
    dff = pd.DataFrame.from_dict(viewData)
    dff = dff.breed.value_counts()
    
    return [
        dcc.Graph(            
            figure = (px.pie(dff, names='breed', title='Preferred Animals')).update_layout(autosize=False, width=800, height=800)
        )    
    ]
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
       'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if viewData is None:
        dff = pd.DataFrame.from_records(db.read({}))
        dff.drop(columns=['_id'],inplace=True)
        row = 0
        return 
    elif index is None:
        row = 0
    else: 
        row = index[0]
        
    return [
        dl.Map(style={'width': '1500px', 'height': '800px'}, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)

Dash app running on http://127.0.0.1:18578/
